In [26]:
import pandas as pd
import numpy as np
import scipy as sp
import xgboost as xgb
import explainer
from sklearn.metrics import recall_score, precision_score, roc_auc_score

In [2]:
# read the data
# The "validation" set plays the role of the test in the competition
# We keep their name validation here
'''
Replace the path here with your data path!
'''
path = '../../Downloads/donation_data/'
train = pd.read_csv(path+'cup98LRN.txt')
val = pd.read_csv(path+'cup98val.txt')
val_label = pd.read_csv(path+'valtargt.txt')

/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,RECINHSE,...,TARGET_D,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,CLUSTER2,GEOCODE2
0,8901,GRI,0,IL,61081,,,3712,0,,...,0.0,0,L,4,E,X,X,X,39.0,C
1,9401,BOA,1,CA,91326,,,5202,0,,...,0.0,0,L,2,G,X,X,X,1.0,A
2,9001,AMH,1,NC,27017,,,0,0,,...,0.0,1,L,4,E,X,X,X,60.0,C
3,8701,BRY,0,CA,95953,,,2801,0,,...,0.0,1,L,4,E,X,X,X,41.0,C
4,8601,,0,FL,33176,,,2001,0,X,...,0.0,1,L,2,F,X,X,X,26.0,A


In [4]:
# merge the label for the validation
val = val.merge(val_label, on='CONTROLN')

In [5]:
#Unfinished
clf_train = train[["AGE", "WEALTH2",  "HIT", "MALEMILI",
         "MALEVET", "VIETVETS", "WWIIVETS", "LOCALGOV", "STATEGOV", "FEDGOV", 
         "ETH7", "ETH10", "ETH11", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6",
         "VC1", "VC2", "VC3", "VC4", "NUMPRM12", "CARDGIFT", "TIMELAG", "AVGGIFT", "LASTGIFT"]]
reg_train = clf_train[train['TARGET_B']==1]
val = val[["AGE",  "WEALTH2",  "HIT", "MALEMILI",
         "MALEVET", "VIETVETS", "WWIIVETS", "LOCALGOV", "STATEGOV", "FEDGOV",  
         "ETH7", "ETH10", "ETH11", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6",
         "VC1", "VC2", "VC3", "VC4", "NUMPRM12", "CARDGIFT", "TIMELAG", "AVGGIFT", "LASTGIFT"]]
train_b = train['TARGET_B']
train_d = train['TARGET_D'][train['TARGET_B']==1]
# val_b = val['TARGET_B']
# val_d = val['TARGET_D']

In [6]:
print(len(reg_train))
print(len(clf_train))
print(len(train_b))
print(len(train_d))

4843
95412
95412
4843


In [20]:
train_b.head()

0    0
1    0
2    0
3    0
4    0
Name: TARGET_B, dtype: int64

In [7]:
# unbalanced dataset
sum(train_b)/len(train_b)

0.05075881440489666

# Training

In [31]:
# For classification and regression, we use xgboost 
gclf = xgb.XGBClassifier()
gclf.fit(clf_train.values, train_b.values)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [39]:
prob = gclf.predict_proba(clf_train)[:,1]
roc_auc_score(train_b, prob)
recall_score(val_b, gclf.predict(val))
precision_score(val_b, gclf.predict(val))

/anaconda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.0

In [51]:
np.percentile(gclf.predict_proba(val.values)[:,1], 95)

0.09054996147751808

In [35]:
# Regressor
greg = xgb.XGBRegressor()
greg.fit(reg_train.values, train_d.values)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [49]:
np.percentile(greg.predict(val.values),95)

33.39040260314941

# Explain classification

In [56]:
np.percentile(gclf.predict_proba(val.values)[:,1]*greg.predict(val.values),95)

1.3688489794731138

In [22]:
default = val.mean()

In [24]:
def scoring_function(data):
    return gclf.predict_proba(data)[:, 1]

In [57]:
# Difference between individual explanations and joint explanations
threshold1 = 0.0905
probs = gclf.predict_proba(val.values)[:,1]
target1 = threshold1 < probs
threshold2 = 33.390
values = greg.predict(val.values)
target2 = threshold2 < values
exps = gclf.predict_proba(val.values)[:,1]*greg.predict(val.values)
threshold3 = 1.368
target3 = threshold3 < exps
target = target1 & target2 & target3
sample_c = val.values[target]

In [59]:
len(sample_c)

21

In [60]:
# Explain classification
def scoring_function(data):
    return gclf.predict_proba(data)[:, 1]
explain1 = explainer.Explainer(scoring_function, default, prune=False)
explanations1 = explain1.explain(sample_c, thresholds=threshold1)

In [61]:
# Explain regression
def scoring_function(data):
    return greg.predict(data)
explain2 = explainer.Explainer(scoring_function, default, prune=False)
explanations2 = explain2.explain(sample_c, thresholds=threshold2)

In [62]:
# Explain expectation
def scoring_function(data):
    return greg.predict(data)*gclf.predict_proba(data)[:, 1]
explain3 = explainer.Explainer(scoring_function, default, prune=False)
explanations3 = explain3.explain(sample_c, thresholds=threshold3)

In [93]:
# Get LaTex table 1
continuous_features = ["AGE",  "WEALTH2",  "HIT", "MALEMILI",
         "MALEVET", "VIETVETS", "WWIIVETS", "LOCALGOV", "STATEGOV", "FEDGOV",  
         "ETH7", "ETH10", "ETH11", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6",
         "VC1", "VC2", "VC3", "VC4", "NUMPRM12", "CARDGIFT", "TIMELAG", "AVGGIFT", "LASTGIFT"]
ix = 0
ix_explanations = explanations1[ix]
print("\\begin{{tabular}}{{{0}}}".format("S"*(1+len(ix_explanations))))
print("\\toprule")
print('&'.join(["{Explanations $\\rightarrow$}"]+ ["{{{0}}}".format(i+ 1) for i in range(len(ix_explanations))]) + "\\\\")
print("\\midrule")
features = np.unique(np.array([item for sublist in ix_explanations for item in sublist]))
for f in features:
    line = ["{{{0}}}".format(continuous_features[f].replace('_', '\_'))]
    for e in ix_explanations:
        if f in e:
            is_high = default[f] < sample_c[ix, f]
            change_needed = "{$\\uparrow$}" if is_high else "{$\\downarrow$}"
            line += [change_needed]
        else:
            line += [""]
    print("&".join(line) + "\\\\")
print("\\midrule") 
print("\\multicolumn{6}{l}{$\\uparrow$ means feature is too large to grant credit.} \\\\")    
print("\\multicolumn{6}{l}{$\\downarrow$ means feature is too small to grant credit.} \\\\")
print("\\bottomrule")    
print("\\end{tabular}")

\begin{tabular}{SSSSSS}
\toprule
{Explanations $\rightarrow$}&{1}&{2}&{3}&{4}&{5}\\
\midrule
{WWIIVETS}&&&{$\uparrow$}&&\\
{VC1}&&&&{$\downarrow$}&\\
{VC3}&{$\uparrow$}&&&&\\
{NUMPRM12}&&{$\uparrow$}&&&\\
{CARDGIFT}&&&&&{$\uparrow$}\\
\midrule
\multicolumn{6}{l}{$\uparrow$ means feature is too large to grant credit.} \\
\multicolumn{6}{l}{$\downarrow$ means feature is too small to grant credit.} \\
\bottomrule
\end{tabular}


In [82]:
# Get LaTex table 2
continuous_features = ["AGE",  "WEALTH2",  "HIT", "MALEMILI",
         "MALEVET", "VIETVETS", "WWIIVETS", "LOCALGOV", "STATEGOV", "FEDGOV",  
         "ETH7", "ETH10", "ETH11", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6",
         "VC1", "VC2", "VC3", "VC4", "NUMPRM12", "CARDGIFT", "TIMELAG", "AVGGIFT", "LASTGIFT"]
ix = 0
ix_explanations = explanations2[ix]
print("\\begin{{tabular}}{{{0}}}".format("S"*(1+len(ix_explanations))))
print("\\toprule")
print('&'.join(["{Explanations $\\rightarrow$}"]+ ["{{{0}}}".format(i+ 1) for i in range(len(ix_explanations))]) + "\\\\")
print("\\midrule")
features = np.unique(np.array([item for sublist in ix_explanations for item in sublist]))
for f in features:
    line = ["{{{0}}}".format(continuous_features[f].replace('_', '\_'))]
    for e in ix_explanations:
        if f in e:
            is_high = default[f] < sample_c[ix, f]
            change_needed = "{$\\uparrow$}" if is_high else "{$\\downarrow$}"
            line += [change_needed]
        else:
            line += [""]
    print("&".join(line) + "\\\\")
print("\\midrule") 
print("\\multicolumn{7}{l}{$\\uparrow$ means feature is too large to grant credit.} \\\\")    
print("\\multicolumn{7}{l}{$\\downarrow$ means feature is too small to grant credit.} \\\\")
print("\\bottomrule")    
print("\\end{tabular}")

\begin{tabular}{SSSSSSS}
\toprule
{Explanations $\rightarrow$}&{1}&{2}&{3}&{4}&{5}&{6}\\
\midrule
{AGE}&&&&{$\downarrow$}&&\\
{STATEGOV}&&&&&{$\downarrow$}&\\
{AFC5}&&&{$\uparrow$}&&&\\
{VC3}&&&&&&{$\uparrow$}\\
{NUMPRM12}&&{$\uparrow$}&&&&\\
{AVGGIFT}&{$\uparrow$}&&&&&\\
{LASTGIFT}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}\\
\midrule
\multicolumn{7}{l}{$\uparrow$ means feature is too large to grant credit.} \\
\multicolumn{7}{l}{$\downarrow$ means feature is too small to grant credit.} \\
\bottomrule
\end{tabular}


In [83]:
# Get LaTex table 3
continuous_features = ["AGE",  "WEALTH2",  "HIT", "MALEMILI",
         "MALEVET", "VIETVETS", "WWIIVETS", "LOCALGOV", "STATEGOV", "FEDGOV",  
         "ETH7", "ETH10", "ETH11", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6",
         "VC1", "VC2", "VC3", "VC4", "NUMPRM12", "CARDGIFT", "TIMELAG", "AVGGIFT", "LASTGIFT"]
ix = 0
ix_explanations = explanations3[ix]
print("\\begin{{tabular}}{{{0}}}".format("S"*(1+len(ix_explanations))))
print("\\toprule")
print('&'.join(["{Explanations $\\rightarrow$}"]+ ["{{{0}}}".format(i+ 1) for i in range(len(ix_explanations))]) + "\\\\")
print("\\midrule")
features = np.unique(np.array([item for sublist in ix_explanations for item in sublist]))
for f in features:
    line = ["{{{0}}}".format(continuous_features[f].replace('_', '\_'))]
    for e in ix_explanations:
        if f in e:
            is_high = default[f] < sample_c[ix, f]
            change_needed = "{$\\uparrow$}" if is_high else "{$\\downarrow$}"
            line += [change_needed]
        else:
            line += [""]
    print("&".join(line) + "\\\\")
print("\\midrule") 
print("\\multicolumn{7}{l}{$\\uparrow$ means feature is too large to grant credit.} \\\\")    
print("\\multicolumn{7}{l}{$\\downarrow$ means feature is too small to grant credit.} \\\\")
print("\\bottomrule")    
print("\\end{tabular}")

\begin{tabular}{SSSSSS}
\toprule
{Explanations $\rightarrow$}&{1}&{2}&{3}&{4}&{5}\\
\midrule
{AGE}&{$\downarrow$}&&&&\\
{WWIIVETS}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}\\
{AFC4}&&&&&{$\uparrow$}\\
{VC1}&&&&&{$\downarrow$}\\
{VC3}&&&{$\uparrow$}&&\\
{NUMPRM12}&&{$\uparrow$}&&&\\
{CARDGIFT}&&&&{$\uparrow$}&\\
{AVGGIFT}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}\\
{LASTGIFT}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}&{$\uparrow$}\\
\midrule
\multicolumn{7}{l}{$\uparrow$ means feature is too large to grant credit.} \\
\multicolumn{7}{l}{$\downarrow$ means feature is too small to grant credit.} \\
\bottomrule
\end{tabular}


In [80]:
train_d.max()

200.0

In [86]:
short = ["AGE",  "WEALTH2",  "HIT", "MALEMILI",
         "MALEVET", "VIETVETS", "WWIIVETS", "LOCALGOV", "STATEGOV", "FEDGOV",  
         "ETH7", "ETH10", "ETH11", "AFC1", "AFC2", "AFC3", "AFC4", "AFC5", "AFC6",
         "VC1", "VC2", "VC3", "VC4", "NUMPRM12", "CARDGIFT", "TIMELAG", "AVGGIFT", "LASTGIFT"]
long =  ["Age",  "Wealth Rating",  "Mail Order Response", "Males active in the Military",
         "Males Veterans", "Vietnam Veterans", "World War two Veterans", "Employed by Local Goverment", "Employed by State Goverment", "Employed by Federal Goverment",  
         "Percent Japanese", "Percent Korean", "Percent Vietnamese", "Percent Adults in Active Military Service", 
                       "Percent Males in Active Military Service", "Percent Females in Active Military Service", 
                       "Percent Adult Veterans Age 16+", "Percent Male Veterans Age 16+", "Percent Female Veterans Age 16+",
         "Percent Vietnam Veterans Age 16+", "Percent Korean Veterans Age 16+", "Percent WW2 Veterans Age 16+", 
                       "Percent Veterans Serving After May 1975 Only", "Number of promotions received in the last 12 months", 
                       "Number of lifetime gifts to card promotions to date ", "Number of months between first and second gift", 
                       "Average dollar amount of gifts to date", "Dollar amount of most recent gift"]

In [87]:
# print latex code
for i in range(len(short)):
    print(long[i]+' ('+short[i]+')'+', ')

Age (AGE), 
Wealth Rating (WEALTH2), 
Mail Order Response (HIT), 
Males active in the Military (MALEMILI), 
Males Veterans (MALEVET), 
Vietnam Veterans (VIETVETS), 
World War two Veterans (WWIIVETS), 
Employed by Local Goverment (LOCALGOV), 
Employed by State Goverment (STATEGOV), 
Employed by Federal Goverment (FEDGOV), 
Percent Japanese (ETH7), 
Percent Korean (ETH10), 
Percent Vietnamese (ETH11), 
Percent Adults in Active Military Service (AFC1), 
Percent Males in Active Military Service (AFC2), 
Percent Females in Active Military Service (AFC3), 
Percent Adult Veterans Age 16+ (AFC4), 
Percent Male Veterans Age 16+ (AFC5), 
Percent Female Veterans Age 16+ (AFC6), 
Percent Vietnam Veterans Age 16+ (VC1), 
Percent Korean Veterans Age 16+ (VC2), 
Percent WW2 Veterans Age 16+ (VC3), 
Percent Veterans Serving After May 1975 Only (VC4), 
Number of promotions received in the last 12 months (NUMPRM12), 
Number of lifetime gifts to card promotions to date  (CARDGIFT), 
Number of months betw